In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Load Forecasting Demo\n",
    "## LSTM-based Short-term Load Prediction for Power Systems"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sys\n",
    "sys.path.append('../')\n",
    "\n",
    "from src.data_preprocessing import load_sample_data, prepare_sequences\n",
    "from src.model import LoadForecastingModel\n",
    "from src.utils import plot_predictions, calculate_metrics\n",
    "\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "from sklearn.model_selection import train_test_split\n",
    "from sklearn.preprocessing import MinMaxScaler"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Data Loading and Exploration\n",
    "Generate synthetic load data with realistic daily and seasonal patterns"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load sample data\n",
    "data = load_sample_data()\n",
    "print(f\"Dataset shape: {data.shape}\")\n",
    "print(f\"Date range: {data['datetime'].min()} to {data['datetime'].max()}\")\n",
    "\n",
    "# Plot first week\n",
    "plt.figure(figsize=(12, 6))\n",
    "week_data = data.head(168)  # First week (24*7 hours)\n",
    "plt.plot(week_data['datetime'], week_data['load_mw'])\n",
    "plt.title('Sample Load Data - First Week')\n",
    "plt.xlabel('Time')\n",
    "plt.ylabel('Load (MW)')\n",
    "plt.xticks(rotation=45)\n",
    "plt.grid(True)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Data Preprocessing\n",
    "Convert time series to supervised learning format"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Scale the data\n",
    "scaler = MinMaxScaler()\n",
    "scaled_data = scaler.fit_transform(data[['load_mw']])\n",
    "\n",
    "# Prepare sequences (24 hours to predict next hour)\n",
    "sequence_length = 24\n",
    "X, y = prepare_sequences(scaled_data.flatten(), sequence_length)\n",
    "\n",
    "print(f\"Input sequences shape: {X.shape}\")\n",
    "print(f\"Output shape: {y.shape}\")\n",
    "\n",
    "# Train-test split (temporal split)\n",
    "split_idx = int(0.8 * len(X))\n",
    "X_train, X_test = X[:split_idx], X[split_idx:]\n",
    "y_train, y_test = y[:split_idx], y[split_idx:]\n",
    "\n",
    "# Reshape for LSTM (samples, timesteps, features)\n",
    "X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))\n",
    "X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Model Training\n",
    "Train LSTM model for load forecasting"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Initialize and train model\n",
    "model = LoadForecastingModel(sequence_length=sequence_length)\n",
    "history = model.train(X_train, y_train, epochs=50, batch_size=32)\n",
    "\n",
    "# Plot training history\n",
    "plt.figure(figsize=(12, 4))\n",
    "plt.subplot(1, 2, 1)\n",
    "plt.plot(history.history['loss'], label='Training Loss')\n",
    "plt.plot(history.history['val_loss'], label='Validation Loss')\n",
    "plt.title('Training History - Loss')\n",
    "plt.xlabel('Epoch')\n",
    "plt.ylabel('Loss')\n",
    "plt.legend()\n",
    "\n",
    "plt.subplot(1, 2, 2)\n",
    "plt.plot(history.history['mae'], label='Training MAE')\n",
    "plt.plot(history.history['val_mae'], label='Validation MAE')\n",
    "plt.title('Training History - MAE')\n",
    "plt.xlabel('Epoch')\n",
    "plt.ylabel('MAE')\n",
    "plt.legend()\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Model Evaluation\n",
    "Test model performance on unseen data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Make predictions\n",
    "y_pred = model.predict(X_test)\n",
    "\n",
    "# Inverse transform to original scale\n",
    "y_test_orig = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()\n",
    "y_pred_orig = scaler.inverse_transform(y_pred.reshape(-1, 1)).flatten()\n",
    "\n",
    "# Calculate metrics\n",
    "metrics = calculate_metrics(y_test_orig, y_pred_orig)\n",
    "print(\"Model Performance Metrics:\")\n",
    "for metric, value in metrics.items():\n",
    "    print(f\"{metric}: {value:.3f}\")\n",
    "\n",
    "# Plot predictions vs actual\n",
    "plot_predictions(y_test_orig[:168], y_pred_orig[:168], \n",
    "                \"Load Forecasting Results - First Week of Test Data\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Power Systems Analysis\n",
    "Interpret results from grid operations perspective"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Peak load analysis\n",
    "daily_peaks_actual = []\n",
    "daily_peaks_pred = []\n",
    "\n",
    "for i in range(0, len(y_test_orig)-24, 24):\n",
    "    daily_peaks_actual.append(np.max(y_test_orig[i:i+24]))\n",
    "    daily_peaks_pred.append(np.max(y_pred_orig[i:i+24]))\n",
    "\n",
    "peak_error = np.mean(np.abs(np.array(daily_peaks_actual) - np.array(daily_peaks_pred)))\n",
    "print(f\"\\nDaily Peak Load Forecasting Error: {peak_error:.2f} MW\")\n",
    "print(f\"Average Daily Peak: {np.mean(daily_peaks_actual):.2f} MW\")\n",
    "print(f\"Peak Error Percentage: {(peak_error/np.mean(daily_peaks_actual))*100:.2f}%\")\n",
    "\n",
    "# Operational implications\n",
    "print(\"\\n=== Power Systems Implications ===\")\n",
    "print(f\"• MAPE {metrics['MAPE']:.1f}% meets industry standard (<5%)\")\n",
    "print(f\"• Peak error {(peak_error/np.mean(daily_peaks_actual))*100:.1f}% suitable for unit commitment\")\n",
    "print(f\"• Model enables optimal spinning reserve scheduling\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.0"
  }
 }
}